# A/B тестирование

Представим, что мы проводили эксперимент на пользователях в течение 2 недель. В результаты мы получили сырые данные об эксперименте, которые нам надо обработать и сделать выводы. 

В результате у нас получилось 2 таблички:

* Ответы наши сервиса с рекомендациями — в них мы знаем, какому пользователю что мы порекомендовали и в какую группу его отнесли. И, конечно, знаем момент времени, когда это произошло.

* Данные о лайках — в них мы знаем, какой пользователь и какой пост лайкнул, в том числе момент времени, когда это произошло.

Загрузим данные и посмотрим на них

In [1]:
import numpy as np
import pandas as pd

views = pd.read_csv('https://drive.google.com/uc?id=1AQNrqqrITmV-Swu834-y66ePinmOXFcs')
likes = pd.read_csv('https://drive.google.com/uc?id=13kQ_ybyReJ-J6DVjASqgZlzfnLUKZi0b')

In [2]:
likes.head()

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833


In [3]:
views.head()

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829


#### Начнём с проверки разбиения групп.

In [4]:
tmp = views.groupby('user_id').exp_group.nunique().reset_index()
tmp[tmp.exp_group > 1]

,user_id,exp_group
10071,25623,2
20633,55788,2
54475,142283,2
57065,148670,2


Всего 4 пользователя, удалим их

In [5]:
bad_users = tmp[tmp.exp_group > 1].user_id.values

views = views[~np.in1d(views.user_id, bad_users)]
likes = likes[~np.in1d(likes.user_id, bad_users)]

Проверим, что группы разбились нормально

In [6]:
views.groupby('user_id').first().exp_group.value_counts(normalize='True')

test       0.502377
control    0.497623
Name: exp_group, dtype: float64

Похоже на правду, но проверим критерием

In [7]:
views.groupby('user_id').first().exp_group.value_counts()

test       32659
control    32350
Name: exp_group, dtype: int64

In [8]:
from scipy.stats import binomtest

binomtest(k=32659, n=32659+32350, p=0.5)

BinomTestResult(k=32659, n=65009, alternative='two-sided', proportion_estimate=0.5023765940100602, pvalue=0.2270501563614752)

#### Попробуем оценить, а улучшаются ли наши метрики в тестовой группе. Посчитаем долю пользователей, которая сделала хотя бы один лайк за время эксперимента без разбивки на группы.

Соберём разбиение на группы

In [9]:
groups = views.groupby('user_id').first().reset_index()[['user_id', 'exp_group']]
groups.head()

,user_id,exp_group
0,200,test
1,201,test
2,202,test
3,212,test
4,213,test


Посчитаем число лайков на пользователя

In [10]:
users_w_likes = (
    likes
    .groupby('user_id')
    .post_id.count()
    .reset_index()
    .rename(columns={'post_id': 'like_num'})
)
users_w_likes.head()

,user_id,like_num
0,200,1
1,201,3
2,202,2
3,212,4
4,213,7


Соберём вместе

In [11]:
tmp = pd.merge(groups, users_w_likes, on='user_id', how='left')
tmp.like_num = tmp.like_num.fillna(0)
tmp['has_like'] = (tmp['like_num'] > 0).astype(int)

In [12]:
tmp.groupby('exp_group')[['has_like', 'like_num']].mean()

,has_like,like_num
exp_group,,
control,0.891314,3.487079
test,0.898160,3.592578


Здесь мы посчитали отдельно по группам, видно что ~ 89.5%

#### Ответим на вопрос "стали люди ставить больше лайков в тестовой группе". Используем уровень значимости 0.05.

In [13]:
from scipy.stats import mannwhitneyu

mannwhitneyu(
    tmp[tmp.exp_group == 'control'].like_num,
    tmp[tmp.exp_group == 'test'].like_num,
)

MannwhitneyuResult(statistic=518358073.0, pvalue=2.9585062792441964e-05)

Обе метрики статистически значимо улучшились, то есть лайкать стали больше

#### Посчитаем hitrate (или долю рекомендаций, в которые пользователи кликнули)

Соединим все пары показом и кликов пользователей (не забудем про показы без кликов)

In [14]:
tmp = pd.merge(views, likes, on='user_id', how='outer')
tmp.post_id = tmp.post_id.fillna(-1).astype(int)

Распарсим сами показы

In [15]:
tmp['recommendations'] = tmp.recommendations.apply(
    lambda x: list(map(int, filter(bool, x[1:-1].split(' '))))
)

Пробежимся по всем показам и сравним с лайками. Пометим то, слишком старое или из будущего, лайки по непоказанным рекомендациям

In [16]:
tmp.post_id = tmp.apply(
    lambda row:
    -1
    if
        (row.post_id == -1) | 
        ((row.timestamp_x > row.timestamp_y) &
         (row.timestamp_x + 60 * 60 < row.timestamp_y)) |
        (row.post_id not in row.recommendations)
    else
    row.post_id, axis=1)

Сравним размерности (предполагаем, что таймстемпы показов у пользователей были уникальны)

In [17]:
len(views), len(tmp)

(193268, 1016889)

In [18]:
views.groupby(['user_id', 'timestamp']).first().shape

(193268, 2)

Кажется, что ок. Давайте уникализируем

In [19]:
def my_agg(values):
    values = set(values)
    if -1 in values and len(values) >= 2:
        return 1
    elif -1 not in values:
        return 1
    return 0

tmp_agg = tmp.groupby(['user_id', 'exp_group', 'timestamp_x']).post_id.agg(my_agg)

In [20]:
# Доля показов, в которых был хотя бы один лайк.
tmp_agg.reset_index().post_id.mean()

0.7143758925429973

#### А теперь  оценим различие между группами и значимость. z-критерий для долей мы здесь не можем применять, так как у нас в каждой выборке один и тот же пользователь может встречаться несколько раз. Применим бакетный подход (то есть перейдём к бакетам и по ним оценим значимость), чтобы поститать групповой hitrate (или CTR) — доля hitrate по группе/бакету. Используем 100 бакетов. Уровень значимости останется тем же на уровне 0.05.

In [21]:
tmp_agg = tmp_agg.reset_index().rename(columns={'post_id': 'hitrate'})

In [22]:
# Подготовим бакеты
import hashlib

tmp_agg['bucket'] = tmp_agg['user_id'].apply(
    lambda x: int(hashlib.md5((str(x) + 'bbb').encode()).hexdigest(), 16) % 100
)

In [23]:
tmp_agg['view'] = 1

new_df = tmp_agg.groupby(['exp_group', 'bucket']).sum(['hitrate', 'view']).reset_index()
new_df['hitrate_new'] = new_df.hitrate / new_df.view

new_df.head()

,exp_group,bucket,user_id,timestamp_x,hitrate,view,hitrate_new
0,control,0,75055655,1472627637383,621,890,0.697753
1,control,1,78264361,1557009869357,668,941,0.709883
2,control,2,89756485,1664574067853,707,1006,0.702783
3,control,3,74901253,1523924072519,646,921,0.701412
4,control,4,77518818,1603347973264,710,969,0.732714


In [24]:
#  Посчитаем метрику
tmp_agg.groupby('exp_group').hitrate.mean()

exp_group
control    0.707741
test       0.720975
Name: hitrate, dtype: float64

Разница довольно большая! Но что со значимостью

In [25]:
from scipy.stats import mannwhitneyu, ttest_ind
mannwhitneyu(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

MannwhitneyuResult(statistic=2452.0, pvalue=4.829847062588435e-10)

In [26]:
ttest_ind(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

Ttest_indResult(statistic=-6.304578169376004, pvalue=1.8432228173570576e-09)

В тестовой группе hitrate выше на 1 п.п. (до бакетирования), есть статистически значимая разница

Проведен A/B эксперимент с моделями, а также проанализировали его. Ответили на самый главный вопрос  (стали ли наши рекомендации лучше), но в реальном продукте могли бы ещё посчитать денежные метрики и более точно посчитать метрики качества рекомендаций.